## Second level scraper code to scrape the details of the restaurants and also the comments of the users

In [1]:
# import necessary libraries
from requests import get
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from random import randint
from time import time 
from IPython.core.display import clear_output
from warnings import warn
import datetime

In [2]:
# retrieve dataframe of links scraped previously
links_df = pd.read_csv('./data/trip-advisor-scraper-main.csv')
links_df.head(10)

,Restaurant Name,Restaurant Type,Webpage
0,1. Positano @ RP,"Italian, European",https://www.tripadvisor.com.sg/Restaurant_Revi...
1,2. Grand Shanghai Restaurant,"Chinese, Asian",https://www.tripadvisor.com.sg/Restaurant_Revi...
2,3. Fu Lin Men (NSRCC),"Chinese, Seafood",https://www.tripadvisor.com.sg/Restaurant_Revi...
3,4. Entre-Nous creperie,"French, European",https://www.tripadvisor.com.sg/Restaurant_Revi...
4,5. NOX - Dine in the Dark,"European, Fusion",https://www.tripadvisor.com.sg/Restaurant_Revi...
5,6. The Mind Cafe,"Cafe, Pub",https://www.tripadvisor.com.sg/Restaurant_Revi...
6,7. Song Garden,"Chinese, Asian",https://www.tripadvisor.com.sg/Restaurant_Revi...
7,8. Fu Lin Men (CSC),"Chinese, Asian",https://www.tripadvisor.com.sg/Restaurant_Revi...
8,9. Melt Cafe,International,https://www.tripadvisor.com.sg/Restaurant_Revi...
9,10. Positano Risto,"Italian, Pizza",https://www.tripadvisor.com.sg/Restaurant_Revi...


In [3]:
# extract the links out
#list(links_df["Webpage"])
# check if length is correct
len(list(links_df["Webpage"]))

1013

In [4]:
# store links as a list
list_of_links = list(links_df["Webpage"])
list_of_links

['https://www.tripadvisor.com.sg/Restaurant_Review-g294265-d21180746-Reviews-Positano_RP-Singapore.html',
 'https://www.tripadvisor.com.sg/Restaurant_Review-g294265-d1145149-Reviews-Grand_Shanghai_Restaurant-Singapore.html',
 'https://www.tripadvisor.com.sg/Restaurant_Review-g294265-d17171783-Reviews-Fu_Lin_Men_NSRCC-Singapore.html',
 'https://www.tripadvisor.com.sg/Restaurant_Review-g294265-d1193730-Reviews-Entre_Nous_creperie-Singapore.html',
 'https://www.tripadvisor.com.sg/Restaurant_Review-g294265-d4611806-Reviews-NOX_Dine_in_the_Dark-Singapore.html',
 'https://www.tripadvisor.com.sg/Restaurant_Review-g294265-d2089644-Reviews-The_Mind_Cafe-Singapore.html',
 'https://www.tripadvisor.com.sg/Restaurant_Review-g294265-d12310362-Reviews-Song_Garden-Singapore.html',
 'https://www.tripadvisor.com.sg/Restaurant_Review-g294265-d13873832-Reviews-Fu_Lin_Men_CSC-Singapore.html',
 'https://www.tripadvisor.com.sg/Restaurant_Review-g294265-d785219-Reviews-Melt_Cafe-Singapore.html',
 'https://www

## Actual Scraping

In [5]:
# get the start value for every new page in yelp
start = [str(i) for i in range(0,501,10)]
# debug
print(start, end=' ')
print(f'\nNumber of pages scraped: {len(start)}')

['0', '10', '20', '30', '40', '50', '60', '70', '80', '90', '100', '110', '120', '130', '140', '150', '160', '170', '180', '190', '200', '210', '220', '230', '240', '250', '260', '270', '280', '290', '300', '310', '320', '330', '340', '350', '360', '370', '380', '390', '400', '410', '420', '430', '440', '450', '460', '470', '480', '490', '500'] 
Number of pages scraped: 51


In [6]:
### RUN THIS CELL TO EMPTY THE LIST!!! ###
# redeclaring lists to store data in

# multiple values
cust_names = []
cust_ratings = []
cust_comments = []

# counter
count = 0

# number of requests
REQUESTS = 2

# flag variable to check the scrape
# if unsuccessful scrape, try again
unsuccessful = True

# preparing the monitoring of the loop
start_time = time()
requests = 1 # start request with request 1

In [7]:
# for every comment in the interval of 10
for link_raw in list_of_links[:3]:
    sleep(10)
    count+=1 # increment count to determine which link it is being scraped
    print(f'----- LINK {count} -----')
    for pageStart in start:

        # Break the loop if the number of requests is greater than expected
        if requests > REQUESTS:
            warn('Number of requests was greater than expected.')
            break # this break is a problem

        unsuccessful = True
        fail_count = 0

        while unsuccessful:
            # make a get request
            #response = get(f'https://www.yelp.com/biz/jumbo-seafood-singapore-4?start={pageStart}')
            #response = get(f'https://www.tripadvisor.com.sg/Restaurant_Review-g294265-d7348336-Reviews-or{pageStart}-Sunday_Folks-Singapore.html')
            link_array = link_raw.split('Reviews-')
            link = link_array[0] + 'Reviews-' + 'or' + str(pageStart) + "-" +link_array[1]
            print(link)
            response = get(link)

            # pause the loop
            sleep(randint(3,5))

            # monitor the requests
            elapsed_time = time() - start_time
            print(f'Request {requests}; Frequency: {requests/elapsed_time} requests/s')
            #clear_output(wait = True)

            # Parse the content of the request with BeautifulSoup
            page_html = BeautifulSoup(response.text, 'html.parser')

            # get the comment container for all 20 comments in a page
            comment_containers = page_html.find_all('div', class_='review-container')

            if len(comment_containers) != 0:
                print(f"Request {requests}: SUCCESS --> Failed Count: {fail_count}")
                unsuccessful = False
            else:
                fail_count+=1
                #print(f"Request {requests}: unsuccessful scrape") # debug
                pass
        requests += 1

        # for every comments in 10
        for com in comment_containers:

            # scrape the customer name
            cust_name = com.find('div', class_='info_text pointer_cursor').text
            cust_names.append(cust_name)

            # scrape the customer ratings
            cust_rating_raw = com.find('div', class_ ='ui_column is-9').span['class']

            # convert the rating to an integer
            cust_rating = 0
            if cust_rating_raw[1] == 'bubble_50':
                cust_rating = 5
            elif cust_rating_raw[1] == 'bubble_40':
                cust_rating = 4
            elif cust_rating_raw[1] == 'bubble_30':
                cust_rating = 3
            elif cust_rating_raw[1] == 'bubble_20':
                cust_rating = 2
            elif cust_rating_raw[1] == 'bubble_10':
                cust_rating = 1
            else:
                cust_rating = 0
            cust_ratings.append(cust_rating)

            # scrape the customer comment
            cust_comment_raw = com.find('div', class_='entry').text

            # get rid of the unwanted characters like "..." and "More"
            comment_ = cust_comment_raw.replace("..."," ")

            # remove instances with "More" tab if there are more words
            cust_comment = ""
            if comment_[-5:] == ".More":
                cust_comment = comment_[:-4]
            else:
                cust_comment = comment_

            cust_comments.append(cust_comment)
            
        # Throw a warning for non-200 status codes
        if response.status_code != 200:
            warn('Request: {}; Status code: {}'.format(requests, response.status_code))

----- LINK 1 -----
https://www.tripadvisor.com.sg/Restaurant_Review-g294265-d21180746-Reviews-or0-Positano_RP-Singapore.html
Request 1; Frequency: 0.05832719278079059 requests/s
Request 1: SUCCESS --> Failed Count: 0
https://www.tripadvisor.com.sg/Restaurant_Review-g294265-d21180746-Reviews-or10-Positano_RP-Singapore.html
Request 2; Frequency: 0.09084621458168182 requests/s
Request 2: SUCCESS --> Failed Count: 0


C:\Users\ValuedAcerCustomer\.conda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Number of requests was greater than expected.
  # Remove the CWD from sys.path while we load stuff.


----- LINK 2 -----


In [ ]:
### BACKUP ###
# redeclaring lists to store data in

# multiple values
cust_names = []
cust_ratings = []
cust_comments = []

# number of requests
REQUESTS = 3

# flag variable to check the scrape
# if unsuccessful scrape, try again
unsuccessful = True

# preparing the monitoring of the loop
start_time = time()
requests = 1 # start request with request 1

# -------------------------------------------------------

# for every comment in the interval of 20
for pageStart in start:
    
    # Throw a warning for non-200 status codes
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))

    # Break the loop if the number of requests is greater than expected
    if requests > REQUESTS:
        warn('Number of requests was greater than expected.')
        break
        
    unsuccessful = True
    fail_count = 0
    
    while unsuccessful:
        # make a get request
        #response = get(f'https://www.yelp.com/biz/jumbo-seafood-singapore-4?start={pageStart}')
        response = get(f'https://www.tripadvisor.com.sg/Restaurant_Review-g294265-d7348336-Reviews-or{pageStart}-Sunday_Folks-Singapore.html')
        
        # pause the loop
        sleep(randint(3,5))

        # monitor the requests
        elapsed_time = time() - start_time
        print(f'Request {requests}; Frequency: {requests/elapsed_time} requests/s')
        #clear_output(wait = True)

        # Parse the content of the request with BeautifulSoup
        page_html = BeautifulSoup(response.text, 'html.parser')

        # get the comment container for all 20 comments in a page
        comment_containers = page_html.find_all('div', class_='review-container')
        
        if len(comment_containers) != 0:
            print(f"Request {requests}: SUCCESS --> Failed Count: {fail_count}")
            unsuccessful = False
        else:
            fail_count+=1
            #print(f"Request {requests}: unsuccessful scrape") # debug
            pass
    requests += 1
    
    # for every comments in 20
    for com in comment_containers:
        
        # scrape the customer name
        cust_name = com.find('div', class_='info_text pointer_cursor').text
        cust_names.append(cust_name)
        
        # scrape the customer ratings
        cust_rating_raw = com.find('div', class_ ='ui_column is-9').span['class']
        
        # convert the rating to an integer
        cust_rating = 0
        if cust_rating_raw[1] == 'bubble_50':
            cust_rating = 5
        elif cust_rating_raw[1] == 'bubble_40':
            cust_rating = 4
        elif cust_rating_raw[1] == 'bubble_30':
            cust_rating = 3
        elif cust_rating_raw[1] == 'bubble_20':
            cust_rating = 2
        elif cust_rating_raw[1] == 'bubble_10':
            cust_rating = 1
        else:
            cust_rating = 0
        cust_ratings.append(cust_rating)
        
        # scrape the customer comment
        cust_comment_raw = com.find('div', class_='entry').text
        
        # get rid of the unwanted characters like "..." and "More"
        comment_ = cust_comment_raw.replace("..."," ")

        # remove instances with "More" tab if there are more words
        cust_comment = ""
        if comment_[-5:] == ".More":
            cust_comment = comment_[:-4]
        else:
            cust_comment = comment_

        cust_comments.append(cust_comment)

In [9]:
for i in range(1,5):
    for j in range(1,6):
        if j==3:
            break
        print(f'i is {i} & j is {j}')

i is 1 & j is 1
i is 1 & j is 2
i is 2 & j is 1
i is 2 & j is 2
i is 3 & j is 1
i is 3 & j is 2
i is 4 & j is 1
i is 4 & j is 2
